<a href="https://colab.research.google.com/github/Vulture-hash/Autoencoders/blob/main/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preprocessing

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist

(X_train, _), (X_test, _) = mnist.load_data()

#Normalize
x_train = X_train.astype('float32')/255.
x_test = X_test.astype('float32')/255.

#Flatten
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

**Vanilla Autoencoder**

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

#Encoder
input_layer = Input(shape=(784, ))
encoder = Dense(64, activation='relu')(input_layer)

#Bottleneck
bottleneck = Dense(32, activation='relu')(encoder)

#Decoder
decoder = Dense(64, activation='relu')(bottleneck)
output_layer= Dense(784, activation='relu')(decoder)

autoencoder = Model(input_layer, output_layer)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()

In [12]:
autoencoder.fit(
    x_train, x_train,
    epochs=25,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test)
)

**Visualizing**

In [15]:
import matplotlib.pyplot as plt
reconstructed = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
  ax = plt.subplot(2, n, i+1)
  plt.imshow(x_test[i].reshape(28, 28))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  #Display reconstruction
  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(reconstructed[i].reshape(28, 28))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()

In [16]:
for layer in autoencoder.layers[-4:]:
  layer.trainable=True

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(
    x_train, x_train,
    epochs=25,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test)
)

**Experimenting Across Different Bottlenecks**

In [22]:
import multiprocessing
from tensorflow.keras import Model, Input, layers as L
import tensorflow as tf

# Define the training function
def train_autoencoder(bottleneck_size):
    input_layer = Input(shape=(784,))
    encoder = L.Dense(64, activation='relu')(input_layer)
    bottleneck = L.Dense(bottleneck_size, activation='relu')(encoder)

    # Decoder
    decoder = L.Dense(64, activation='relu')(bottleneck)
    output_layer = L.Dense(784, activation='sigmoid')(decoder)

    autoencoder = Model(input_layer, output_layer)

    # Compile and fit the model
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.fit(
        x_train, x_train,
        epochs=50,
        batch_size=256,
        shuffle=True,
        validation_data=(x_test, x_test)
    )

    return autoencoder

# Bottleneck sizes to test
bottleneck_sizes = [16, 32, 64]

for bottleneck in bottleneck_sizes:
      autoencoder = train_autoencoder(bottleneck)
      loss = autoencoder.evaluate(x_test, x_test)
      print(f"Bottleneck size {bottleneck} - Test loss: {loss}")


**Using Regularization**

In [17]:
from tensorflow.keras.regularizers import L2
from tensorflow import keras
def train_autoencoder():
    input_layer = Input(shape=(784,))
    encoder = L.Dense(64,kernel_regularizer=L2(0.001),activation='relu')(input_layer)
    bottleneck = L.Dense(32, kernel_regularizer=L2(0.001),activation='relu')(encoder)

    # Decoder
    decoder = L.Dense(64, kernel_regularizer=L2(0.001),activation='relu')(bottleneck)
    output_layer = L.Dense(784, kernel_regularizer=L2(0.001),activation='sigmoid')(decoder)

    autoencoder = Model(input_layer, output_layer)

    # Compile and fit the model
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.fit(
        x_train, x_train,
        epochs=50,
        batch_size=256,
        shuffle=True,
        validation_data=(x_test, x_test)
    )

    return autoencoder

train_autoencoder()